#### Заметки:
1. Переставить порядок кода на следующий:
    * [Подготовка StimViz](#StimViz) (30 минут)
    * [Объявление метода change_TMS_effects(x, y, z).](#change_TMS_effects) Без конечной части про формирование цвета
    * [Подготовка ГА](#GA_preparation)
    * [Запуск ГА](#GA) (долго)
    * Визуализация результата через StimViz
2. Стоит подумать над ускорением алгоритма (либо через уменьшение этапов в ГА, либо через уменьшение времени выполнения расчета на каждом этапе. Или все вместе)
3. Распарарелить вычисления

 ## <a name="StimViz"></a> **I.** Подготовка StimViz

In [3]:
from stimvis.tms_stimulation import (my_deriv,
                                    load_elems,
                                    get_field,
                                    get_ttrd,
                                    deriv_e_field,
                                    simulation)
# Проверить: так ли необходимы все импорты?!
from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti
import numpy as np

from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.tracking import streamline
import copy
import pickle

import simnibs
from simnibs import sim_struct, run_simnibs

In [4]:
%%time
# Ввод данных и начало трансформации, связанной с различием систем координат
tractography_file='../StimVis_Data/tracts.trk'

sft = load_trk(tractography_file, tractography_file)
streams = sft.streamlines
streams_array = np.asarray(streams)
print('imported tractography data:'+tractography_file)

fname_T1 = '../StimVis_Data/T1fs_conform.nii.gz'#input("Please, specify the T1fs_conform image that has been generated during head meshing procedure. ")
data_T1, affine_T1 = load_nifti(fname_T1)

fname_FA = '../StimVis_Data/dti_fa.nii' #input("Please, specify the FA image. ")
data_FA, affine_FA = load_nifti(fname_FA)
print('loaded T1fs_conform.nii and FA images')

global mesh_path
mesh_path = '../StimVis_Data/subject.msh' #input("Please, specify the head mesh file. ")

last_slach = max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
global subject_name
subject_name = mesh_path[last_slach:-4]


# specify the directory where you would like to save your simulation results
# input example:'/home/Example_data/Output'
global out_dir
out_dir = '../StimVis_Data/Output' #input("Please, specify the directory where you would like to save your simulation results. ")
out_dir = out_dir+'/simulation_at_pos_'

imported tractography data:../StimVis_Data/tracts.trk
loaded T1fs_conform.nii and FA images
Wall time: 1.33 s


In [5]:
%%time 
#29:33
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1, data_FA, affine_FA)
print('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                    affine_T1, affine_FA,
                    starting_affine=starting_affine)
print('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0, 
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images.
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                         affine_T1, affine_FA,
                         starting_affine=starting_affine)

print('calculated Affine 3D transform')

identity = np.eye(4)

inv_affine_FA = np.linalg.inv(affine_FA)
inv_affine_T1 = np.linalg.inv(affine_T1)
inv_affine = np.linalg.inv(affine.affine)

# transforming streamlines to FA space
new_streams_FA = streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array = np.asarray(new_streams_FA)

T1_to_FA = np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1))
FA_to_T1 = np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
global new_streams_T1_array
new_streams_T1_array = np.asarray(new_streams_T1)

# calculating amline derivatives along the streamlines to get the local orientation of the streamlines
global streams_array_derivative
streams_array_derivative = copy.deepcopy(new_streams_T1_array)

print('calculating streamline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam = new_streams_T1_array[stream]
    for t in range(len(my_steam[:, 0])):
        streams_array_derivative[stream][t, 0] = my_deriv(t, my_steam[:, 0])
        streams_array_derivative[stream][t, 1] = my_deriv(t, my_steam[:, 1])
        streams_array_derivative[stream][t, 2] = my_deriv(t, my_steam[:, 2])
        deriv_norm = np.linalg.norm(streams_array_derivative[stream][t, :])
        streams_array_derivative[stream][t, :] = streams_array_derivative[stream][t, :]/deriv_norm    

calculated c_of_mass transformation
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated 3D translation transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Rigid 3D transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Affine 3D transform
calculating streamline derivatives
Wall time: 30min 8s




## <a name="change_TMS_effects"></a>**II.** Объявление def change_TMS_effects(x, y, z)


In [79]:
def change_TMS_effects(x, y, z):
    
    l1 = 2  # membrane space constant 2mm
    l2 = l1**2
    print(x,y,z)
    effect_max = 0.100
    effect_min = -0.100
    position = [x-256/2, y-256/2, z-256/2]  # -256/2 because of a freesurfer RAS coordinate system
    current_out_dir = out_dir+str(x)+'_'+str(y)+'_'+str(z)

    simulation(mesh_path, current_out_dir, pos_centre=position)

    mesh_file = current_out_dir+'/'+subject_name+'_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = simnibs.msh.read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

    effective_field = copy.deepcopy(new_streams_T1_array)

    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        print('starting _'+str(stream)+' out of '+str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            # -256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0]-256/2
            y = my_steam[t, 1]-256/2
            z = my_steam[t, 2]-256/2
            xyz = np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t, 0] = l1*np.dot(field_vector_xyz, streams_array_derivative[stream][t, :])
            effective_field[stream][t, 1] = l2*deriv_e_field(xyz, field_at_nodes, streams_array_derivative[stream][t, :], ttt)
            effective_field[stream][t, 2] = effective_field[stream][t, 0] + effective_field[stream][t, 1]
            if (effective_field[stream][t, 2] < effect_min):
                effect_min = effective_field[stream][t, 2]
            if effective_field[stream][t, 2] > effect_max:
                effect_max = effective_field[stream][t, 2]
    with open(current_out_dir+'/'+subject_name+'_effective_field.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()

    #-----------------Подсчет эффекта------
    sum_effect=0;
    for stream in range(len(effective_field)):
        effects=effective_field[stream][:,2]
        sum_effect += max([abs(effects[i:i+3].mean()) for i in range(len(effects))])
    return sum_effect

-----------
## <a name="GA_preparation"></a> **III.** Подготовка ГА
Генетический алгоритм со сферической системой координат и двоичным кодированием углов

In [80]:
import random
import math
import numpy as np

In [81]:
def calc_fitness(angles):
    if angles[0]>180 || angles[1]>180:
        return 0
    x,y,z = get_coords_from_angles(angles)
    return change_TMS_effects(x, y, z)

In [77]:
calc_fitness([100,0])

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
INFO:simnibs:Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0
INFO:simnibs:Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0


254.05539238556264 105.77303325863292 128.0


[ simnibs ]INFO: Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0
INFO:simnibs:Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position f

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_254.05539238556264_105.77303325863292_128.0\subject_TMS_1-0001_Magstim_70mm_Fig8_ni

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


1.1275809481298626

In [1]:
# Создание класса особи
def get_coords_from_angles(angles): #0-255
    r=256/2
    a= math.radians(angles[0])
    b= math.radians(angles[1])
    # необходимо также добавить сдвиг центра окружности для привидения координат
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    return (x,y,z)

class Individual:
    def __init__(self, angles):
        self.angles=angles
        self.fitness=calc_fitness(angles)
    def __lt__(self, other):
         return self.fitness > other.fitness
    def mutation(self, shift):
        self.angles[random.getrandbits(1)]^= 1 << shift
        self.fitness=calc_fitness(self.angles)

In [97]:
#def 2.1 Объединение в пару
def get_parents_id():
    def get_parent(fit_values):
        randnum=np.random.random()
        fitness_sum = fit_values.sum() #sum(fit_values)
        fit_values= fit_values/fitness_sum
        cdf=np.concatenate([np.array([0]), fit_values.cumsum()])
        for i in range(len(cdf)-1):
            if((randnum>cdf[i])&(randnum<=cdf[i+1])):
                return i
            
    #generation.sort()
    fit_values = np.array([x.fitness for x in  generation])
    parent_id_a= get_parent(fit_values)
    parent_id_b= get_parent(np.delete(fit_values,parent_id_a))
    if parent_id_b>=parent_id_a:
        parent_id_b+=1;
    return(parent_id_a, parent_id_b)
#def get_parents_id(n):
#    return random.randint(0,n), random.randint(0,n)

In [84]:
# def 2.2 Скрещивание
def crossover(parents_id):
    angles_a= generation[parents_id[0]].angles
    angles_b= generation[parents_id[1]].angles
    сhromosome_a= bin(angles_a[0])[2:].zfill(8) + bin(angles_a[1])[2:].zfill(8)
    сhromosome_b= bin(angles_b[0])[2:].zfill(8) + bin(angles_b[1])[2:].zfill(8)
    
    crossover_point= random.randint(0,14)
    q=0
    children = ['', '']
    for i in range(16): # alfa, beta ∈ [0;180] => необходимо байт на кодирование каждого угла, а это 16 битов
        children[q%2] += сhromosome_a[i]
        children[(q+1)%2] += сhromosome_b[i]
        if(i==crossover_point): q+=1
    #from binary to int
    return ([int(children[0][:7],2), int(children[0][8:],2)],
            [int(children[1][:7],2), int(children[1][8:],2)])

## <a name="GA"></a>**IV.** Выполнение ГА:

In [86]:
%%time
#1
cnst_n_start=12
cnst_n=22
cnst_n_child=7
cnst_mut_start=3
cnst_nochng_steps=10

generation=[Individual([random.randint(0,180) for i in range(2)])
            for x in range(cnst_n_start)]
generation.sort()

step=0
max_fitness=generation[0].fitness
nochanges_steps_counter=0
print('max_fitness in the first generation is ', max_fitness)

while(not(step==8)):#(nochanges_steps_counter>cnst_nochng_steps and max_fitness>-7 or max_fitness==0)): #(step==8)):#
    #2 Разбиение на пары и создание потомков
    i=0
    while (i<cnst_n_child):
        children=crossover(get_parents_id()) #(get_parents_id(len(generation)-1)) #
        for x in children:
            if(x not in [z.angles for z in generation]):
                generation.append(Individual(x))
                i+=1
    #3 Селекция
    generation.sort()
    generation= generation[:cnst_n]
    # Мутация
    if(nochanges_steps_counter>cnst_mut_start):
        generation[random.randint(5,len(generation)-1)].mutation(random.randint(0,7))
    #4
    step+=1; nochanges_steps_counter+=1
    if(max_fitness<generation[0].fitness):
        max_fitness=generation[0].fitness
        nochanges_steps_counter=0;
print('steps:',step)
print('----------')

print(generation[0].angles, max_fitness)

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
INFO:simnibs:Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136
INFO:simnibs:Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136


192.824055525985 23.148538331009036 162.4675617217136


[ simnibs ]INFO: Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136
INFO:simnibs:Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calcula

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_192.824055525985_23.148538331009036_162.4675617217136\subject_T

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


...

108.1249413389553 242.04883509611108 182.60627488155635


[ simnibs ]INFO: Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_108.1249413389553_242.04883509611108_182.60627488155635
INFO:simnibs:Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_108.1249413389553_242.04883509611108_182.60627488155635
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Cal

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_108.1249413389553_242.04883509611108_182.60627488155635\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_108.1249413389553_242.04883509611108_182.60627488155635\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_108.1249413389553_242.04883509611108_182.60627488155635\sub

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
max_fitness in the first generation is  6.61154233973836


TypeError: 'numpy.float64' object is not callable

-------------
Дальше различный мусор

In [106]:
%%time
# Продолжить алгоритм (+5)
while(not(step==5)):#(nochanges_steps_counter>cnst_nochng_steps and max_fitness>-7 or max_fitness==0)): #(step==8)):#
    #2 Разбиение на пары и создание потомков
    i=0
    while (i<cnst_n_child):
        children=crossover(get_parents_id()) #(get_parents_id(len(generation)-1)) #
        for x in children:
            if(x not in [z.angles for z in generation]):
                generation.append(Individual(x))
                i+=1
    #3 Селекция
    generation.sort()
    generation= generation[:cnst_n]
    # Мутация
    if(nochanges_steps_counter>cnst_mut_start):
        generation[random.randint(5,len(generation)-1)].mutation(random.randint(0,7))
    #4
    #print(step)
    step+=1; nochanges_steps_counter+=1
    if(max_fitness<generation[0].fitness):
        max_fitness=generation[0].fitness
        nochanges_steps_counter=0;
    print("\033[34m STEP: {}" .format(step))
print('steps:',step)
print('----------')

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
INFO:simnibs:Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218
INFO:simnibs:Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218


106.77572405730365 239.95132251384265 186.31321889683218


[ simnibs ]INFO: Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218
INFO:simnibs:Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:C

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_106.77572405730365_239.95132251384265_186.31321889683218\

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
INFO:simnibs:Head Mesh: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104
INFO:simnibs:Simulation Folder: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104


67.98999445435177 241.01729188594265 131.1449911254104


[ simnibs ]INFO: Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104
INFO:simnibs:Running simulations in the directory: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: C:\ProgramData\Anaconda3\envs\simnibs_env\lib\site-packages\simnibs\ccd-files\Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calcu

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104\subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: C:\Users\МосинРоманВасильевич\Documents\SimNIBS\StimVis_Data\Output\simulation_at_pos_67.98999445435177_241.01729188594265_131.1449911254104\subjec

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
 STEP: 5
steps: 5
----------
Wall time: 2h 5min 8s


In [161]:
print(len(generation), step)
print(generation[0].angles,generation[0].fitness)
print('hell:', cnst_n_start+ cnst_n_child* step)

36 8808
[73, 68] -3.7230966501055365
hell: 176172


In [107]:
for x in generation:
    print(x.angles, x.fitness)

[100, 92] 6.61154233973836
[72, 86] 6.324387411430825
[59, 86] 4.178484995644902
[73, 69] 4.151513923204855
[114, 92] 3.8380385652441156
[117, 69] 3.501941083852558
[58, 69] 3.445269798233358
[58, 65] 3.2888343983033907
[79, 62] 3.272000520214788
[50, 92] 2.7561911164659527
[48, 110] 2.676289923848845
[50, 62] 2.614809596226632
[29, 110] 2.550076206070203
[39, 158] 2.516201181561415
[42, 94] 2.4572279124239134
[27, 110] 2.350721669230002
[36, 94] 2.2525318498248246
[28, 177] 1.0015915904155688
[25, 110] 2.1344108731694815
[29, 81] 2.111571078901506
[29, 86] 2.111571078901506
[39, 92] 2.051750297550539


In [110]:
with open('../_generation.txt', 'wb') as f:
    pickle.dump(generation, f)
f.close()

In [88]:
get_coords_from_angles([100,92])

(123.6007302491076, 105.77303325863292, 253.97860284642405)